<a href="https://colab.research.google.com/github/romerocruzsa/cp-anemia-detection/blob/capstone-benchmark/notebooks/capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install torchinfo
# !pip install torchmetrics

In [2]:
# Import necessary libraries for file handling, data manipulation, and visualization
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from timm import create_model

# Import libraries for working with images and transformations
from PIL import Image
import cv2 as cv

# Import PyTorch modules for model building, data handling, and evaluation
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import torchvision.models.quantization as quant_models
from torch.utils.checkpoint import checkpoint
from torch.utils.data import Dataset, DataLoader, Subset
# from torchinfo import summary

# Import libraries for machine learning metrics and model evaluation
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, mean_squared_error, mean_absolute_error, r2_score
# import torchmetric
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
import gc

# Set the seed.
seed = 42
torch.manual_seed(seed)

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
# data_dir="/workspace/cp-anemia-detection/data/cp-anemia/"
# weights_dir="/workspace/cp-anemia-detection/data/notebooks/weights/"

data_dir = "/content/drive/MyDrive/CAWT_Sebastian_202425/CP-AnemiC/"
weights_dir = "/content/drive/MyDrive/CAWT_Sebastian_202425/Weights/"
anemic_dir=data_dir+"/Anemic/"
non_anemic_dir=data_dir+"/Non-anemic/"
signature = "02042024"

In [5]:
data_sheet_path = data_dir+"Anemia_Data_Collection_Sheet.csv"
data_sheet = pd.read_csv(data_sheet_path)
display(data_sheet)

,IMAGE_ID,HB_LEVEL,Severity,Age(Months),GENDER,REMARK,HOSPITAL,CITY/TOWN,MUNICIPALITY/DISTRICT,REGION,COUNTRY
0,Image_001,9.80,Moderate,6,Female,Anemic,Nkawie-Toase Government Hospital,Nkawie-Toase,Atwima Nwabiagya South,Ashanti,Ghana
1,Image_002,9.90,Moderate,24,Male,Anemic,Ejusu Government Hospital,Ejusu,Ejusu Municipality,Ashanti,Ghana
2,Image_003,11.10,Non-Anemic,24,Female,Non-anemic,Ahmadiyya Muslim Hospital,Tachiman,Techiman Municipality,Bono-East,Ghana
3,Image_004,12.50,Non-Anemic,12,Male,Non-anemic,Ahmadiyya Muslim Hospital,Tachiman,Techiman Municipality,Bono-East,Ghana
4,Image_005,9.90,Moderate,24,Male,Anemic,Sunyani Municipal Hospital,Sunyani,Sunyani Municipality,Bono,Ghana
...,...,...,...,...,...,...,...,...,...,...,...
705,Image_706,12.80,Non-Anemic,48,Male,Non-anemic,Bolgatanga Regional Hospital,Bolgatanga,Bolgatanga Municipality,Upper East,Ghana
706,Image_707,11.47,Non-Anemic,48,Female,Non-anemic,Ahmadiyya Muslim Hospital,Tachiman,Techiman Municipality,Bono-East,Ghana
707,Image_708,11.60,Non-Anemic,60,Male,Non-anemic,Komfo Anokye Teaching Hospital,Kumasi,Kumasi Metropolitan,Ashanti,Ghana
708,Image_709,12.10,Non-Anemic,48,Male,Non-anemic,Bolgatanga Regional Hospital,Bolgatanga,Bolgatanga Municipality,Upper East,Ghana


In [6]:
# Mapping diagnosis to severity
severity_mapping = {
    "Non-Anemic": 0,
    "Mild": 1,
    "Moderate": 2,
    "Severe": 3,
}

data_sheet['Severity'] = data_sheet['Severity'].map(severity_mapping)
display(data_sheet)

,IMAGE_ID,HB_LEVEL,Severity,Age(Months),GENDER,REMARK,HOSPITAL,CITY/TOWN,MUNICIPALITY/DISTRICT,REGION,COUNTRY
0,Image_001,9.80,2,6,Female,Anemic,Nkawie-Toase Government Hospital,Nkawie-Toase,Atwima Nwabiagya South,Ashanti,Ghana
1,Image_002,9.90,2,24,Male,Anemic,Ejusu Government Hospital,Ejusu,Ejusu Municipality,Ashanti,Ghana
2,Image_003,11.10,0,24,Female,Non-anemic,Ahmadiyya Muslim Hospital,Tachiman,Techiman Municipality,Bono-East,Ghana
3,Image_004,12.50,0,12,Male,Non-anemic,Ahmadiyya Muslim Hospital,Tachiman,Techiman Municipality,Bono-East,Ghana
4,Image_005,9.90,2,24,Male,Anemic,Sunyani Municipal Hospital,Sunyani,Sunyani Municipality,Bono,Ghana
...,...,...,...,...,...,...,...,...,...,...,...
705,Image_706,12.80,0,48,Male,Non-anemic,Bolgatanga Regional Hospital,Bolgatanga,Bolgatanga Municipality,Upper East,Ghana
706,Image_707,11.47,0,48,Female,Non-anemic,Ahmadiyya Muslim Hospital,Tachiman,Techiman Municipality,Bono-East,Ghana
707,Image_708,11.60,0,60,Male,Non-anemic,Komfo Anokye Teaching Hospital,Kumasi,Kumasi Metropolitan,Ashanti,Ghana
708,Image_709,12.10,0,48,Male,Non-anemic,Bolgatanga Regional Hospital,Bolgatanga,Bolgatanga Municipality,Upper East,Ghana


In [ ]:
# Define data augmentations or transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=np.random.rand()),
    transforms.RandomVerticalFlip(p=np.random.rand()),
    transforms.RandomRotation(degrees=np.random.randint(0, 360)),
    transforms.RandomAffine(degrees=np.random.randint(0, 360)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom dataset class
class CPAnemiCDataset(Dataset):
    def __init__(self, dir, df, transform=None):
        self.dir = dir
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_id = row['IMAGE_ID']
        img_folder = row['REMARK']
        img_path = os.path.join(self.dir, img_folder, img_id + ".png")
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        multiclass_label = torch.tensor(row['Severity'])
        hb_level = torch.tensor(row['HB_LEVEL'])

        return img, multiclass_label, hb_level

    # Load the dataset
image_dataset = CPAnemiCDataset(data_dir, data_sheet, transform=transform)
train_dataset, test_dataset = train_test_split(image_dataset, test_size=0.20, shuffle=True)

print(f"Image Dataset Size (All): {len(image_dataset)}, \
        Train Size: {len(train_dataset)}, \
        Test Size: {len(test_dataset)}")

BATCH_SIZE = 32
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

In [ ]:
# Default device
device = torch.device('cpu')

# Check for CUDA availability
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    print("CUDA is not available, using CPU.")

print(f"Selected device: {device}")

In [ ]:
def get_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    model_size = "Model Size: %.2f MB" %(os.path.getsize("tmp.pt")/1e6)
    os.remove('tmp.pt')
    return model_size

def checkpointed_forward(model, img):
    """Applies checkpointing and logs GPU latency"""
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)

    start_event.record()  # Start measuring
    class_pred, reg_pred = torch.utils.checkpoint.checkpoint(model, img)
    end_event.record()  # Stop measuring

    torch.cuda.synchronize()  # Ensure timings are recorded correctly
    latency = start_event.elapsed_time(end_event)

    return class_pred, reg_pred, latency

# Static Weighting Function. Set eta_class to desired importance (Classification > .5, Regression < .5, Equal == .5)
def sw_loss(loss_class, loss_reg, eta_class=0.5):
    eta_reg = 1 - eta_class
    total_loss = (eta_class * loss_class) + (eta_reg * loss_reg)
    return total_loss

In [ ]:
def train(dataloader, model, class_loss, reg1_loss, reg2_loss, optimizer):
    model.train()
    total_loss = 0
    total_ce_loss = 0
    total_mse_loss = 0
    total_mae_loss = 0
    correct = 0
    total_samples = 0

    for _, (img, multiclass, hb_level) in enumerate(dataloader):
        img = img.to(device)
        multiclass = multiclass.to(device).long()  # Ensure correct type for CrossEntropyLoss
        hb_level = hb_level.to(device).unsqueeze(1).float()

        optimizer.zero_grad()

        # Forward pass
        class_pred, reg_pred, latency = checkpointed_forward(model, img)

        # Compute losses
        ce_loss = class_loss(class_pred, multiclass)  # CrossEntropy for classification
        mse_loss = reg1_loss(reg_pred, hb_level)  # MSE for regression
        mae_loss = reg2_loss(reg_pred, hb_level)  # MAE/L1 for regression
        loss = sw_loss(ce_loss, mse_loss, 0.7)  # Weighted loss

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Track total losses
        total_loss += loss.item()
        total_ce_loss += ce_loss.item()
        total_mse_loss += mse_loss.item()
        total_mae_loss += mae_loss.item()

        # Compute classification accuracy
        class_probs = F.softmax(class_pred, dim=1)
        highest_prob_class = torch.argmax(class_probs, dim=1)
        correct += (highest_prob_class == multiclass).sum().item()
        total_samples += multiclass.size(0)

    avg_loss = total_loss / len(dataloader)
    avg_ce_loss = total_ce_loss / len(dataloader)
    avg_mse_loss = total_mse_loss / len(dataloader)
    avg_mae_loss = total_mae_loss / len(dataloader)
    classification_accuracy = correct / total_samples

    return avg_loss, classification_accuracy, avg_mae_loss, avg_ce_loss, avg_mse_loss

In [ ]:
def eval(dataloader, model, class_loss, reg1_loss, reg2_loss):
    model.eval()
    total_loss = 0
    total_ce_loss = 0
    total_mse_loss = 0
    total_mae_loss = 0
    correct = 0
    total_samples = 0
    total_mae = 0

    torch.cuda.empty_cache()
    gc.collect()

    with torch.no_grad():
        for _, (img, multiclass, hb_level) in enumerate(dataloader):
            img = img.to(device)
            multiclass = multiclass.to(device).long()  # Ensure correct type for CrossEntropyLoss
            hb_level = hb_level.to(device).unsqueeze(1).float()

            # Forward pass
            class_pred, reg_pred, latency = checkpointed_forward(model, img)
            print("Inference Time: {latency} ms")

            # Compute losses
            ce_loss = class_loss(class_pred, multiclass)  # CrossEntropy for classification
            mse_loss = reg1_loss(reg_pred, hb_level)  # MSE for regression
            mae_loss = reg2_loss(reg_pred, hb_level)  # MAE/L1 for regression
            loss = sw_loss(ce_loss, mse_loss, 0.7)  # Weighted loss

            # Track total losses
            total_loss += loss.item()
            total_ce_loss += ce_loss.item()
            total_mse_loss += mse_loss.item()
            total_mae_loss += mae_loss.item()

            # Compute classification accuracy
            class_probs = F.softmax(class_pred, dim=1)
            highest_prob_class = torch.argmax(class_probs, dim=1)
            correct += (highest_prob_class == multiclass).sum().item()
            total_samples += multiclass.size(0)

    avg_loss = total_loss / len(dataloader)
    avg_ce_loss = total_ce_loss / len(dataloader)
    avg_mse_loss = total_mse_loss / len(dataloader)
    avg_mae_loss = total_mae_loss / len(dataloader)
    classification_accuracy = correct / total_samples

    return avg_loss, classification_accuracy, avg_mae_loss, avg_ce_loss, avg_mse_loss

In [ ]:
class MultiModel(nn.Module):
    MODEL_MAPPING = {
        "mobilenetv2": lambda: models.mobilenet_v2(pretrained=False),
        "resnet18": lambda: models.resnet18(pretrained=False),
        "densenet121": lambda: models.densenet121(pretrained=False),
        "vgg16": lambda: models.vgg16(pretrained=False),
        "vit-tiny": lambda: create_model("vit_tiny_patch16_224", pretrained=False),
        "convnext-tiny": lambda: models.convnext_tiny(pretrained=False),
        "efficientnet-b0": lambda: models.efficientnet_b0(pretrained=False),
        "shufflenetv2-0.5x": lambda: models.shufflenet_v2_x0_5(pretrained=False),
        "regnety-400mf": lambda: models.regnet_y_400mf(pretrained=False),
        "mnasnet0_5": lambda: models.mnasnet0_5(pretrained=False),
        "ghostnetv2": lambda: create_model('ghostnetv2_100.in1k', pretrained=False),
        "tinynet-a": lambda: create_model("tinynet_a.in1k", pretrained=False)
    }

    FEATURE_LAYER_MAPPING = {
        "fc": ["resnet", "shufflenet", "regnet"],
        "classifier": ["densenet", "vgg", "mobilenet", "efficientnet",
                       "mnasnet","convnext", "ghostnet", "tinynet"],
        "head": ["vit"]
    }

    def __init__(self, model_name):
        super().__init__()
        self.model_name = model_name.lower()

        if self.model_name not in self.MODEL_MAPPING:
            raise ValueError(f"Model {model_name} not supported")

        self.model = self.MODEL_MAPPING[self.model_name]()
        num_ftrs = self._get_feature_size()

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(num_ftrs, 128),
            nn.ReLU(),
            nn.Linear(128, 5)
        )

        self._assign_classifier()
        print(f"Modified Backbone {get_model_size(self.model)}\n")

    def _get_feature_size(self):
        """Retrieve the number of input features for the last layer."""
        feature_layers = {
            "fc": getattr(self.model, "fc", None),
            "classifier": getattr(self.model, "classifier", None),
            "head": getattr(self.model, "head", None)
        }

        for key, layer in feature_layers.items():
            if layer:
                return layer[-1].in_features if isinstance(layer, nn.Sequential) else layer.in_features

        return getattr(self.model, "num_features", None)

    def _assign_classifier(self):
        """Assigns the appropriate classifier to the model based on its architecture."""
        for attr, models in self.FEATURE_LAYER_MAPPING.items():
            if any(m in self.model_name for m in models):
                setattr(self.model, attr, self.classifier)
                return

    def forward(self, x):
        output = self.model(x)
        return output[:, :4], output[:, 4]  # Class probabilities and Hb level estimate

In [ ]:
models_list = ["mobilenetv2", "resnet18", "densenet121", "vgg16", "vit-tiny",
               "efficientnet-b0", "shufflenetv2-0.5x", "regnety-400mf",
               "mnasnet0_5", "convnext-tiny", "ghostnetv2", "tinynet-a"]
for arch in models_list:
    print(f"Loading model: {arch}")
    model = MultiModel(arch)#.to(device)
    # print(summary(model))
    # print(model)

In [ ]:
BATCH_SIZE = 32
EPOCHS = 1
FOLDS = 5

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cross_entropy_loss = torch.nn.CrossEntropyLoss()  # Multi-class classification loss
mse_loss = torch.nn.MSELoss()  # Regression loss
mae_loss = torch.nn.L1Loss()  # Regression loss

# 5-Fold Cross Validation
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)

# Directory to save the best model
weights_dir = "weights"
os.makedirs(weights_dir, exist_ok=True)

for arch in models_list:
  print(f"Training Model: {arch}")
  # Initialize model and loss functions
  model = MultiModel(arch).to(device)
  # print(f"\n{summary(model)}")
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

  best_val_acc = -float('inf')  # Track best validation accuracy
  train_metrics_df = []
  val_metrics_df = []

  # Training loop
  for epoch in range(EPOCHS):
      print(f"Epoch {epoch+1}/{EPOCHS}")
      fold = 1

      for train_idx, val_idx in kf.split(range(len(image_dataset))):  # FIX: Ensure correct splitting
          train_subset = Subset(image_dataset, train_idx)
          val_subset = Subset(image_dataset, val_idx)

          train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
          val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

          if fold == FOLDS:
              # Validation phase
              avg_val_loss, val_acc, val_mae_loss, val_ce_loss, val_mse_loss = eval(val_loader, model, cross_entropy_loss, mse_loss, mae_loss)

              print(f"Validation: Fold {fold} - Total Loss: {avg_val_loss:.4f}, Accuracy: {val_acc:.4f}, CrossEntropy: {val_ce_loss:.4f}, MSE: {val_mse_loss:.4f}, MAE: {val_mae_loss:.4f}")

              # Save model with the best validation accuracy
              if val_acc > best_val_acc:
                  best_val_acc = val_acc
                  val_metrics_dict = {"Loss": avg_val_loss, "Accuracy": val_acc}
                  val_metrics_df.append(val_metrics_dict)
                  torch.save(model.state_dict(), f"{weights_dir}/model_best_accuracy_{arch}_{signature}.pth")
                  print(f"Best model saved with Accuracy: {best_val_acc:.4f}")

          else:
              # Training phase
              avg_train_loss, train_acc, train_mae_loss, train_ce_loss, train_mse_loss = train(train_loader, model, cross_entropy_loss, mse_loss, mae_loss, optimizer)

              print(f"Training: Fold {fold} - Total Loss: {avg_train_loss:.4f}, Accuracy: {train_acc:.4f}, CrossEntropy: {train_ce_loss:.4f}, MSE: {train_mse_loss:.4f}, MAE: {train_mae_loss:.4f}")

              train_metrics_dict = {"Loss": avg_train_loss, "Accuracy": train_acc, "CrossEntropy": train_ce_loss, "MSE":train_mse_loss}
              train_metrics_df.append(train_metrics_dict)

          fold += 1  # Move to next fold

  # Ensure `get_model_size()` exists or remove this line
  print(f"\nFine-tuned {get_model_size(model)}\n")

In [ ]:
for arch in models_list:
  print(f"{arch}")
  model = MultiModel(arch).to(device)
  model.load_state_dict(torch.load(f"{weights_dir}/model_best_accuracy_{arch}_{signature}.pth"))
  print(f"Fine-tuned {get_model_size(model)}")
  avg_test_loss, test_acc, test_mae_loss, test_ce_loss, test_mse_loss = eval(test_loader, model, loss_fn_class, loss_fn_reg)
  print(f"Testing: Total Loss: {avg_test_loss:.4f}, Accuracy: {test_acc:.4f}, CrossEntropy: {test_ce_loss:.4f}, MSE: {test_mse_loss:.4f}, MAE: {test_mae_loss:.4f}\n")
  %timeit avg_test_loss, test_acc, test_mae_loss, test_ce_loss, test_mse_loss = eval(test_loader, model, loss_fn_class, loss_fn_reg)
  print("")
